In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
from tqdm import tqdm
import os
import data_utils
import model_utils
from attack_utils import get_CSMIA_case_by_case_results, CSMIA_attack, LOMIA_attack
from data_utils import oneHotCatVars, filter_random_data_by_conf_score
from vulnerability_score_utils import get_vulnerability_score, draw_hist_plot
from experiment_utils import MIAExperiment
from disparity_inference_utils import get_confidence_array, draw_confidence_array_scatter, get_indices_by_group_condition, get_corr_btn_sens_and_out_per_subgroup, get_slopes, get_angular_difference, calculate_stds, get_mutual_info_btn_sens_and_out_per_subgroup
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network._base import ACTIVATIONS
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import roc_curve, auc, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.decomposition import PCA
from sklearn.inspection import permutation_importance
from fairlearn.metrics import equalized_odds_difference, demographic_parity_difference
import matplotlib.pyplot as plt
import seaborn as sns
import tabulate
import pickle
# import utils
import copy

import matplotlib as mpl

# Setting the font family, size, and weight globally
mpl.rcParams['font.family'] = 'DejaVu Sans'
mpl.rcParams['font.size'] = 8
mpl.rcParams['font.weight'] = 'light'

In [4]:
experiment = MIAExperiment(sampling_condition_dict_list = 
    {
            'subgroup_col_name': 'ST',
    }, shortname = f"Corr_btn_sens_and_output_for_ST_ranging_from_0_to_-0.5"
)

  2%|▏         | 1/51 [00:00<00:46,  1.07it/s]

before scaling: 505 392 1.288265306122449
after scaling: 392 392 1.0


  4%|▍         | 2/51 [00:01<00:43,  1.13it/s]

before scaling: 496 125 3.968
after scaling: 125 125 1.0


  8%|▊         | 4/51 [00:03<00:40,  1.17it/s]

...

before scaling: 265 171 1.5497076023391814
after scaling: 171 171 1.0


 98%|█████████▊| 50/51 [00:41<00:00,  1.22it/s]

before scaling: 746 326 2.2883435582822087
after scaling: 326 326 1.0


100%|██████████| 51/51 [00:42<00:00,  1.20it/s]

before scaling: 255 87 2.9310344827586206
after scaling: 87 87 1.0
[1000, 1000, 776, 252, 1000, 1000, 1000, 520, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 469, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 751, 1000, 371, 1000, 1000, 1000, 1000, 1000, 735, 1000, 710, 1000, 1000, 1000, 1000, 952, 400, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 715, 1000, 1000, 746, 337, 1000, 508, 1000, 1000, 1000, 821, 1000, 1000, 1000, 905, 1000, 1000, 1000, 1000, 738, 359, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 841, 1000, 1000, 509, 253, 1000, 1000, 1000, 1000, 1000, 1000, 450, 360, 1000, 1000, 1000, 1000, 947, 645, 1000, 1000, 436, 341]


In [5]:
save_model = True

print(f"Training classifier for experiment: {experiment}")
try:
    experiment.clf = model_utils.load_model(f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_target_model.pkl')
    print(f"Loaded classifier for experiment from file: {experiment}")
except:
    # clf = model_utils.get_model(max_iter=500, hidden_layer_sizes=(256, 256))
    experiment.clf = model_utils.get_model(max_iter=500)
    experiment.clf.fit(experiment.X_train, experiment.y_tr_onehot)

    if save_model:
        model_utils.save_model(experiment.clf, f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_target_model.pkl')

Training classifier for experiment: Census19_subgroup_col_name_ST
Loaded classifier for experiment from file: Census19_subgroup_col_name_ST


In [6]:
save_model=True
print(f"Training classifier for experiment: {experiment}")
try:
    experiment.clf = model_utils.load_model(f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_target_model_only_on_test_dummy.pkl')
    print(f"Loaded classifier for experiment from file: {experiment}")
except:
    # clf = model_utils.get_model(max_iter=500, hidden_layer_sizes=(256, 256))
    base_model = model_utils.get_model(max_iter=500)
    experiment.clf = copy.deepcopy(base_model)
    experiment.clf.fit(experiment.X_test, experiment.y_te_onehot)

    if save_model:
        model_utils.save_model(experiment.clf, f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_target_model_only_on_test_dummy.pkl')

Training classifier for experiment: Census19_subgroup_col_name_ST
Loaded classifier for experiment from file: Census19_subgroup_col_name_ST


In [7]:
from sklearn.neural_network import MLPClassifier

class MLPClassifierFC(MLPClassifier):
    def fit(self, X, y, sample_weight=None):
        """
            Fit the model to the given data.
        """
        if sample_weight is not None:
            # resample data according to sample weights
            n_samples = X.shape[0]
            sample_weight = np.asarray(sample_weight)/np.sum(sample_weight)
            sample_idxs = np.random.choice(n_samples, n_samples, p=sample_weight)
            X = X.iloc[sample_idxs]
            y = y[sample_idxs]
            
        return super().fit(X, y)

clf2 = MLPClassifierFC(max_iter=500)
clf2.coefs_ = experiment.clf.coefs_
clf2.intercepts_ = experiment.clf.intercepts_

In [8]:
from fairlearn.reductions import ExponentiatedGradient, DemographicParity, EqualizedOdds, ErrorRate

subgroup_col_name = 'ST'

In [9]:
subgroup_oh_cols = [f'ST_{i}' for i in range(51)]
subgroup_vals_tr = experiment.X_train[subgroup_oh_cols].to_numpy().argmax(axis=1)
subgroup_vals_tr = experiment.X_train[f'ST_0'].to_numpy().ravel()

In [10]:
subgroup_vals_tr

array([1., 1., 1., ..., 0., 0., 0.])

In [11]:
clf3 = MLPClassifierFC(max_iter=500, verbose=True)
clf3.coefs_ = experiment.clf.coefs_
clf3.intercepts_ = experiment.clf.intercepts_
mitigator_db_p01 = ExponentiatedGradient(clf3, EqualizedOdds(), max_iter=10)
mitigator_db_p01.fit(experiment.X_train, experiment.y_tr, sensitive_features=subgroup_vals_tr)

Iteration 1, loss = 0.51777835
Iteration 2, loss = 0.46912649
Iteration 3, loss = 0.46075360
Iteration 4, loss = 0.45556304
Iteration 5, loss = 0.45216112
Iteration 6, loss = 0.44827510
Iteration 7, loss = 0.44533778
Iteration 8, loss = 0.44203308
Iteration 9, loss = 0.43929611
Iteration 10, loss = 0.43665200
Iteration 11, loss = 0.43431058
Iteration 12, loss = 0.43138366
Iteration 13, loss = 0.42942338
Iteration 14, loss = 0.42701461
Iteration 15, loss = 0.42588661
Iteration 16, loss = 0.42364137
Iteration 17, loss = 0.42146638
Iteration 18, loss = 0.42008516
Iteration 19, loss = 0.41843438
Iteration 20, loss = 0.41707000
Iteration 21, loss = 0.41563445
Iteration 22, loss = 0.41398079
Iteration 23, loss = 0.41290925
Iteration 24, loss = 0.41139400
Iteration 25, loss = 0.41042029
Iteration 26, loss = 0.40962343
Iteration 27, loss = 0.40849378
Iteration 28, loss = 0.40795414
Iteration 29, loss = 0.40606990
Iteration 30, loss = 0.40500490
Iteration 31, loss = 0.40375501
Iteration 32, los

ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.EqualizedOdds object at 0x31979fdc0>,
                      estimator=MLPClassifierFC(max_iter=500, verbose=True),
                      max_iter=10, nu=0.0006424673718166643)

In [17]:
model_utils.save_model(mitigator_db_p01, f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_mitigator_eqodds_db_p01_test_{subgroup_col_name}.pkl')

In [14]:
mitigator_db_p01 = model_utils.load_model(f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_mitigator_eqodds_db_p01_test_{subgroup_col_name}.pkl')

In [18]:
clf4 = MLPClassifierFC(max_iter=500)
clf4.coefs_ = experiment.clf.coefs_
clf4.intercepts_ = experiment.clf.intercepts_
mitigator_dp_db_p01 = ExponentiatedGradient(clf4, DemographicParity(difference_bound=0.01))
mitigator_dp_db_p01.fit(experiment.X_test, experiment.y_te, sensitive_features=experiment.X_test[f'{subgroup_col_name}_0'])

ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.DemographicParity object at 0x32458b8b0>,
                      estimator=MLPClassifierFC(max_iter=500),
                      nu=0.0008548056047151522)

In [16]:
get_CSMIA_case_by_case_results(mitigator_db_p01, experiment.X_test, experiment.y_te, experiment.ds, subgroup_col_name='SEX', sensitive_col_name='MAR_1')

,1,0,Overall
Case 1,4145 (65.121),5573 (71.0687),68.5056
Case 2,17798 (55.1002),16661 (66.8666),60.8624
Case 3,3057 (56.3745),2766 (56.0854),56.2350
Case All Cases,25000 (56.8906),25000 (66.5318),61.7476


In [27]:
y_tr_pred = mitigator_db_p01.predict(experiment.X_train)
subgroup_vals_tr = experiment.X_train[f'SEX_0'].to_numpy().ravel()
equalized_odds_difference(experiment.y_tr.ravel(), y_tr_pred, sensitive_features=subgroup_vals_tr)

0.06296000000000002

In [28]:
y_te_pred = mitigator_db_p01.predict(experiment.X_test)
subgroup_vals_te = experiment.X_test[f'SEX_0'].to_numpy().ravel()
equalized_odds_difference(experiment.y_te.ravel(), y_te_pred, sensitive_features=subgroup_vals_te)

0.017199999999999993

In [18]:
y_te_pred = mitigator_db_p01.predict(experiment.X_test)
subgroup_vals_te = experiment.X_test[f'SEX_0'].to_numpy().ravel()
demographic_parity_difference(experiment.y_te.ravel(), y_te_pred, sensitive_features=subgroup_vals_te)

0.014400000000000024

In [28]:
subgroup_vals_te = experiment.X_test[f'{subgroup_col_name}_1'].to_numpy().ravel()
sens_pred_LOMIA = LOMIA_attack(experiment, mitigator_db_p01, experiment.X_test, experiment.y_te, experiment.ds.ds.meta)

In [29]:
correct_indices_LOMIA = (sens_pred_LOMIA == experiment.X_test[[f'{experiment.ds.ds.meta["sensitive_column"]}_1']].to_numpy().ravel())

In [32]:
print(100 * (correct_indices_LOMIA[np.where(subgroup_vals_te==0)[0]].mean()-correct_indices_LOMIA[np.where(subgroup_vals_te==1)[0]].mean()))

11.895999999999995


In [38]:
from model_utils import predict_proba_for_mitiagtor
subgroup_vals_tr = experiment.X_train[f'{subgroup_col_name}_1'].to_numpy().ravel()
male_indices = np.where(subgroup_vals_tr==0)[0]
female_indices = np.where(subgroup_vals_tr==1)[0]
y_tr_pred = np.argmax(mitigator_db_p01._pmf_predict(experiment.X_train), axis=1)

In [39]:
100 * accuracy_score(experiment.y_tr.ravel()[male_indices], y_tr_pred[male_indices])

70.724

In [40]:
100 * accuracy_score(experiment.y_tr.ravel()[female_indices], y_tr_pred[male_indices])

70.3

In [ ]:
mitigator_db_p01

In [ ]:
print(pd.DataFrame.from_dict(imputation_perf_dict, orient='index').T[[100, 500, 1000, 5000]].round(2).to_latex(float_format="%.2f"))

\begin{tabular}{lrrrr}
\toprule
 & 100 & 500 & 1000 & 5000 \\
\midrule
0.100000 & 47.96 & 54.41 & 60.15 & 62.54 \\
0.200000 & 57.79 & 68.95 & 66.93 & 67.68 \\
0.300000 & 64.16 & 71.50 & 70.58 & 71.57 \\
0.400000 & 71.47 & 71.98 & 72.89 & 73.31 \\
0.500000 & 68.84 & 73.43 & 71.82 & 73.46 \\
\bottomrule
\end{tabular}



In [12]:
get_CSMIA_case_by_case_results(mitigator, experiment.X_test, experiment.y_te, experiment.ds, subgroup_col_name='SEX', sensitive_col_name='MAR_1')

,1,0,Overall
Case 1,3750 (65.9039),5272 (69.5954),68.1096
Case 2,18077 (52.4326),16885 (67.0515),59.3924
Case 3,3173 (57.1429),2843 (58.2051),57.6786
Case All Cases,25000 (55.0063),25000 (66.4534),60.7494


In [13]:
get_CSMIA_case_by_case_results(experiment.clf, experiment.X_test, experiment.y_te, experiment.ds, subgroup_col_name='SEX', sensitive_col_name='MAR_1')

,1,0,Overall
Case 1,1998 (63.3056),4075 (63.8072),63.6483
Case 2,19357 (55.7876),17947 (74.5631),65.0548
Case 3,3645 (58.75),2978 (55.9896),57.3698
Case All Cases,25000 (56.9703),25000 (69.6637),63.5705
